<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Linux Introduction: Job Control</h1>  

©2000-24 Chris Seddon 
</div>

Before we get started w need to discuss the concepts of foreground and background processes.  

In the early days of Unix, users only had a single terminal to work with and this made it difficult to run several jobs concurrently.  To overcome this limitation foreground and background processes were introduced.

Actually these terms are confusing.  Technically a process is either running, suspended or terminated.  However the terms foreground and background are how the parent process, in our case `bash`, sees the process.  The terms as some people think, have nothing to do with any output from the process, but simply whether or not `bash` waits for the process to complete before it issues its prompt for the next command:

To wait or not to wait? - that is the question.

First, here is the `bash` shell launching a foreground procees.  `bash` will wait for the command to complete before we can enter the next command:

In [ ]:
sleep 10
echo "foreground process has completed"

Contrast that with a background process.  

To create a background process you add a `&` at the end of the command.  The `bash` shell will not wait for the command to complete - it will issue a prompt and read the next command immediately.

In [ ]:
sleep 10 &
echo "background process has completed"

Now that is out of the way, you can run the `ps` command (process status) to see all jobs running on your machine.

In [ ]:
ps -ef

The `ps` command has several columns.  To see them again we can use head:

In [ ]:
ps -ef | head -1

The columns are as follows: 

| Heading | Meaning |  
|:---|:---|  
UID | user id |
| PID | process id |
| PPID | parent process id|
| C | % cpu utilization |
| STIME | time process started |
| TTY | controlling terminal |
| TIME | cpu time used |
| CMD | command |  

Moving on, now run 3 commands in the background.  I'm using `xeyes` because its easy to see when its been suspended (stopped).  If you haven't played with `xeyes` before, just try moving the mouse near the graphic.

In [ ]:
xeyes&
xeyes&
xeyes&

`ps` is often used with `grep` to filter out the commands we are not interested in:

In [ ]:
ps -ef | grep xeyes

Note that we also pick up the `grep` command.  If that annoys you, a little pattern matching will fix it:

In [ ]:
ps -ef | grep [x]eyes

Alternatively, we can use `pgrep`:

In [ ]:
pgrep -l xeyes

You can kill multiple commands by name with `killall`:

In [ ]:
pkill xeyes
ps -ef | grep [x]eyes

`ps` confirms the xeyes commands have been terminated.  

Let's restart the `xeyes` and try some other commands.

In [ ]:
xeyes&
xeyes&
xeyes&

`jobs` shows which processes are running in an immediate sub-shell.  The `+` and `-` indicate the last 2 processes we interacted with.

In [ ]:
jobs

We can kill a job by requesting the kernel send a TERM signal to a process.  We need to specify the process's pid or its local pid.  Local pids are the ones reported by `jobs`.  Use a `%` sign to distinguish a local pid from a normal pid:

In [ ]:
kill -TERM %1
jobs

We can also suspend (stop) a job by sending a TSTP signal via the kernel.  

This will freeze the `xeyes`.

In [ ]:
kill -TSTP %3

To restart the job use:

In [ ]:
kill -CONT %3
jobs

Note that the `kill` command doesn't always kill a process.  `kill` delivers a signal to a process via the Linux kernel. In the early days of Unix, KILL was the only signal available, hence the naming of the command.  Nowadays there are lots of possible signals, so the command would be better called "send-signal".  

Incidentally, each signal has a number, dependent on version of operating system.  You can use that number in place of the signal name.  Thus these commands are the same:
<pre>
kill -TERM %3
kill -15 %3
</pre>

To get a list of possible signals we can look in the manual pages:

In [ ]:
man signal

Let's kill off all the jobs:

In [ ]:
pkill xeyes
jobs

When we send a signal to a process it is delivered by the Linux kernel.  A process is allowed to ignore the signal if it so chooses.  I've been using `xeyes` because it is a well  behaved process and will obey the signals you send. 

I've written a short script file that doesn't respond to signals called:
<pre>
badly-behaved-process
</pre>
I'm sending it signals periodically.  

When we try to kill it (TERM signal), suspend it (TSTP signal) or interrupt it (INT signal) it just ignores the signals (note the shell thinks it has terminated, but `jobs` confirms it is still running).

So how can we terminate this process?

In [ ]:
cd resources
badly-behaved-process &
jobs
sleep 5
kill -TERM %1

jobs
kill -INT %1
sleep 5

jobs
sleep 5
kill -TSTP %1
sleep 1

jobs

To cater for this situation, two signals are special and can't be ignored.  If you send a KILL signal (always signal 9) the kernel will terminate the process immediately unconditionally.  The STOP signal always suspends a process; the kernel enforces this.  In both cases the signals never arrive at the target process.

So let's kill off our waywood process:

In [ ]:
jobs
kill -9 %1
jobs

We can also bring jobs to the foreground or send to the backgound.  Unfortunately this is difficult to demonstate in a Jupyter Notebook because we are not talking directly to the `bash` shell.  

When we move a background job to the forground, the Jupyter kernel will not allow us to proceed to the next cell.  You will see a `*` on the left of the cell indicating its still running.  In a real `bash` shell we can hit control-C (^C) to terminate a foreground process.  But, in the notebook you'll have to simulate that by hitting the black square on the menu bar above the notebook cells.

You'll need to do this or the notebook will freeze after we issue the next command, which starts a `xeyes` process in the background and then brings it to the foreground.  

Alternatively you could have killed the `xeyes` by clicking the `x` on the graphic.  Notice that when you hit the black square, the Jupyter notebook kills the job.

In [ ]:
xeyes&
fg %1

Its also possible to suspend a foreground job using cotrol-Z (^Z), but again we can't easily demonstrate this in the notebook.  

However we can run a seperate `xterm` and you can run `xeyes` in that.  This will be a running a real `bash` interpreter; this will allow you to try out the ^Z and ^C signals directly.

Let's kill off all our `xeyes` processes and start the `xterm`:

In [ ]:
pkill xeyes
xterm -fg black -bg white -fa 'Monospace' -fs 14 &

Now type the following command in the `xterm`.
<pre>
xeyes
</pre>
`xeyes` will run in the foreground.  

Now type ^Z in the `xterm` to suspend `xeyes`.  Verify it is frozen.  
type
<pre>
jobs
</pre>
to confirm its suspended.  

Now type
<pre>
fg
</pre>
and `xeyes` is back in the foreground and no longer frozen.  Note the `bash` doesn't issue a prompt whilst the foreground process is running.  You may need to click on its parent window to bring it to the front of the screen.

Type ^Z again to suspend `xeyes` and then send it to the background with
<pre>
bg
jobs
</pre>
It will now be active once again.  Moving a command between background and foreground doesn't affect the process - its active (running) in both cases; it only affects whether or not the `bash` shell issues a prompt.

Finally, bring the job to the foreground with
<pre>
fg
</pre>
and kill it with ^C.  You can now kill off the `xterm` window as well.  

That's all folks!